In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from spot_loader import load_spot

Get Prod histo per techno and region

In [8]:
def load_eco2mix(start, end, limit=100):
    BASE_URL = "https://odre.opendatasoft.com/api/explore/v2.1/catalog/datasets/eco2mix-regional-cons-def/records"
    all_rows = []
    offset = 0
    while True:
        params = {
            "where": f"date_heure >= '{start.isoformat()}' AND date_heure < '{end.isoformat()}'",
            "select": "date_heure,libelle_region,eolien,solaire",
            "limit": limit,
            "offset": offset
        }
        resp = requests.get(BASE_URL, params=params)
        resp.raise_for_status()
        data = resp.json()
        rows = data.get("results", [])
        if not rows:
            break
        all_rows.extend(rows)
        offset += limit
        if len(rows) < limit:
            break
    df = pd.DataFrame(all_rows)
    

    df = df.drop_duplicates(
        subset=['date_heure', 'libelle_region'],
        keep='first'
    )

    df_pretraitement = df.copy()

    df['Datetime'] = pd.to_datetime(df['date_heure']).dt.tz_localize(None)
    df['eolien'] = pd.to_numeric(df['eolien'], errors='coerce')
    df['solaire'] = pd.to_numeric(df['solaire'], errors='coerce')
    df = df.drop(columns=['date_heure'])
    df['Datetime'] = df['Datetime'].dt.floor('H')
    df_hourly = df.groupby(['Datetime','libelle_region'], as_index=False).agg({'eolien':'sum','solaire':'sum'})
    return df_pretraitement, df_hourly

In [9]:
def load_eco2mix_by_days(start, end):
    current = start
    all_pretraitement = []
    all_hourly = []
    while current < end:
        next_day = current + timedelta(days=1)
        df_pre, df_hourly = load_eco2mix(current, next_day)
        all_pretraitement.append(df_pre)
        all_hourly.append(df_hourly)
        current = next_day
    df_pretraitement = pd.concat(all_pretraitement, ignore_index=True)
    df_hourly = pd.concat(all_hourly, ignore_index=True)
    return df_pretraitement, df_hourly

start = datetime(2021,1,1)
end = datetime(2021,5,31)
df_pretraitement, df_5months = load_eco2mix_by_days(start, end)

spot_histo = load_spot(start, end)
spot_histo['Datetime'] = pd.to_datetime(spot_histo['Datetime'])

merged = df_5months.merge(spot_histo, on='Datetime', how='left')
merged['year'] = merged['Datetime'].dt.year
merged['month'] = merged['Datetime'].dt.month
merged['eolien_sans_hpsn'] = merged['eolien'].where(merged['Spot']>0,0)
merged['solaire_sans_hpsn'] = merged['solaire'].where(merged['Spot']>0,0)

monthly_region = []
for (year, month, region), g in merged.groupby(['year','month','libelle_region']):
    se = g['eolien'].sum()
    ss = g['solaire'].sum()
    se_s = g['eolien_sans_hpsn'].sum()
    ss_s = g['solaire_sans_hpsn'].sum()
    spot_avg = g['Spot'].mean()
    ve = (g['eolien']*g['Spot']).sum()
    vs = (g['solaire']*g['Spot']).sum()
    ve_s = (g['eolien_sans_hpsn']*g['Spot']).sum()
    vs_s = (g['solaire_sans_hpsn']*g['Spot']).sum()
    cr_e = ve/(se*spot_avg) if se and spot_avg else None
    cr_s = vs/(ss*spot_avg) if ss and spot_avg else None
    cr_e_s = ve_s/(se_s*spot_avg) if se_s and spot_avg else None
    cr_s_s = vs_s/(ss_s*spot_avg) if ss_s and spot_avg else None

    monthly_region.append({
        'Année': year,
        'Mois': month,
        'Région': region,
        'Prod Eolien': se,
        'Prod Eolien sans hpsn' :se_s,                  # <-- add total production
        'CR Eolien': cr_e,
        'CR Eolien sans HPSN': cr_e_s,
        'CR Solaire': cr_s,
        'CR Solaire sans HPSN': cr_s_s,
        'Spot moyen': spot_avg
    })

df_monthly_region = pd.DataFrame(monthly_region)



C:\Users\A220925\AppData\Local\Temp\ipykernel_30824\617308904.py:36: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df['Datetime'] = df['Datetime'].dt.floor('H')
C:\Users\A220925\AppData\Local\Temp\ipykernel_30824\617308904.py:36: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df['Datetime'] = df['Datetime'].dt.floor('H')
C:\Users\A220925\AppData\Local\Temp\ipykernel_30824\617308904.py:36: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df['Datetime'] = df['Datetime'].dt.floor('H')
C:\Users\A220925\AppData\Local\Temp\ipykernel_30824\617308904.py:36: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df['Datetime'] = df['Datetime'].dt.floor('H')
C:\Users\A220925\AppData\Local\Temp\ipykernel_30824\617308904.py:36: FutureWarning: 'H' is deprecated and will be removed in a future versio